<a href="https://colab.research.google.com/github/longnguyen1224/Google-Docs-Markdown-Formatter/blob/main/Software_Engineer_Assessment_Task_Ansible_Health.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import library


In [17]:
import re
import markdown
from google.colab import auth
from googleapiclient.discovery import build
from google.auth import default

Authenticate and Set up Goole Docs API

In [18]:
def authenticate_google():
    """Authenticate with Google APIs in Colab, handling potential errors."""
    try:
        auth.authenticate_user()
        creds, _ = default()
        return build("docs", "v1", credentials=creds)
    except Exception as e:
        print(f"Error during authentication: {e}")
        return None

In [19]:
def create_google_doc(service, title):
    """Create a new Google Doc and return its ID, handling API errors."""
    try:
        doc = service.documents().create(body={"title": title}).execute()
        return doc["documentId"]
    except Exception as e:
        print(f"Error creating Google Doc: {e}")
        return None

Convert to google docs

In [33]:
# Convert Markdown meeting notes into a well-formatted Google Doc, handling API errors.
def convert_markdown_to_google_docs(service, doc_id, markdown_text):
    """Convert Markdown meeting notes into a well-formatted Google Doc."""
    try:
      lines = markdown_text.split("\n")
      requests = []

      # Track current document length dynamically
      current_index = 1

      for line in lines:
          line = line.strip()

          # Convert main title to Heading 1
          if line.startswith("# "):
              text = line[2:] + "\n"
              requests.append({"insertText": {"location": {"index": current_index}, "text": text}})
              current_index += len(text)  # Update position after insertion
              requests.append({
                  "updateParagraphStyle": {
                      "range": {"startIndex": current_index - len(text), "endIndex": current_index},
                      "paragraphStyle": {"namedStyleType": "HEADING_1"},
                      "fields": "namedStyleType"
                  }
              })
          # Convert section headers to Heading 2
          elif line.startswith("## "):
              text = line[3:] + "\n"
              requests.append({"insertText": {"location": {"index": current_index}, "text": text}})
              current_index += len(text)
              requests.append({
                  "updateParagraphStyle": {
                      "range": {"startIndex": current_index - len(text), "endIndex": current_index},
                      "paragraphStyle": {"namedStyleType": "HEADING_2"},
                      "fields": "namedStyleType"
                  }
              })
          # Convert sub-section headers to Heading 3
          elif line.startswith("### "):
              text = line[4:] + "\n"
              requests.append({"insertText": {"location": {"index": current_index}, "text": text}})
              current_index += len(text)
              requests.append({
                  "updateParagraphStyle": {
                      "range": {"startIndex": current_index - len(text), "endIndex": current_index},
                      "paragraphStyle": {"namedStyleType": "HEADING_3"},
                      "fields": "namedStyleType"
                  }
              })
           # Convert markdown checkboxes to Google Docs checkboxes
          elif line.startswith("- [ ] "):
              text = "☐ " + line[6:] + "\n"
              requests.append({"insertText": {"location": {"index": current_index}, "text": text}})
              current_index += len(text)
          # Highlight mentions (@name)
          elif re.search(r"@\w+", line):
              text = line + "\n"
              requests.append({"insertText": {"location": {"index": current_index}, "text": text}})
              current_index += len(text)
          elif line:
              text = line + "\n"
              requests.append({"insertText": {"location": {"index": current_index}, "text": text}})
              current_index += len(text)

      # Apply all changes at once
      service.documents().batchUpdate(documentId=doc_id, body={"requests": requests}).execute()
      print("Google Doc successfully created and formatted.")
    except Exception as e:
      print(f"Error updating Google Doc: {e}")


In [34]:
if __name__ == "__main__":
    markdown_notes = """# Product Team Sync - May 15, 2023
    ## Attendees
    - Sarah Chen (Product Lead)
    - Mike Johnson (Engineering)
    - Anna Smith (Design)
    - David Park (QA)

    ## Agenda
    ### 1. Sprint Review
    * Completed Features
      * User authentication flow
      * Dashboard redesign
      * Performance optimization
        * Reduced load time by 40%
        * Implemented caching solution
    * Pending Items
      * Mobile responsive fixes
      * Beta testing feedback integration

    ### 2. Current Challenges
    * Resource constraints in QA team
    * Third-party API integration delays
    * User feedback on new UI
      * Navigation confusion
      * Color contrast issues

    ### 3. Next Sprint Planning
    * Priority Features
      * Payment gateway integration
      * User profile enhancement
      * Analytics dashboard
    * Technical Debt
      * Code refactoring
      * Documentation updates

    ## Action Items
    - [ ] @sarah: Finalize Q3 roadmap by Friday
    - [ ] @mike: Schedule technical review for payment integration
    - [ ] @anna: Share updated design system documentation
    - [ ] @david: Prepare QA resource allocation proposal

    ## Next Steps
    * Schedule individual team reviews
    * Update sprint board
    * Share meeting summary with stakeholders

    ## Notes
    * Next sync scheduled for May 22, 2023
    * Platform demo for stakeholders on May 25
    * Remember to update JIRA tickets

    ---
    **Meeting recorded by:** Sarah Chen
    **Duration:** 45 minutes"""

    service = authenticate_google()
    if service:
        doc_id = create_google_doc(service, "Product Team Sync Notes")
        if doc_id:
            convert_markdown_to_google_docs(service, doc_id, markdown_notes)


Google Doc successfully created and formatted.
